In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import progressbar
import PIL.Image as Image
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
!unzip testset3.zip | wc -l

In [ ]:
MODEL = "model_v4.pt"
TEST_DATASET = "/content/testset3/"

In [ ]:
uniqueCars = ['chevrolet corvette', 'porsche 911 Turbo', 'ford mustang GT']

hashMap = {}

def EvaluateModel(model):
    hashMap["classic_cars"] = {"correct" : 0, "incorrect" : 0}
    widgets = ['[', progressbar.Timer(format= 'elapsed time: %(elapsed)s'), ']',
      progressbar.Bar('*'),
    ]
  
    bar = progressbar.ProgressBar(max_value=292, widgets=widgets).start()
    correct = 0.0
    total = 0.0
    count = 0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            fname, _ = testloader.dataset.samples[i]
            
            file_data = fname.split('/')
            CLASS, IMAGE_DATA = file_data[-2], file_data[-1]
            IMAGE_DATA = IMAGE_DATA.split("_")[:-1]

            #labels = torch.Tensor([int(data_class[CLASS.lower()])])
  
            count += 1
            # assigning to GPU
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if (predicted == labels.sum()) == 1:
              for category in IMAGE_DATA:
                if category not in hashMap:
                  hashMap[category] = {"correct" : 0, "incorrect" : 0}
                hashMap[category]['correct'] += 1

              if CLASS in uniqueCars:
                hashMap["classic_cars"]["correct"] += 1

            else:
              for category in IMAGE_DATA:
                if category not in hashMap:
                  hashMap[category] = {"correct" : 0, "incorrect" : 0}
                hashMap[category]['incorrect'] += 1
              
              if CLASS in uniqueCars:
                hashMap["classic_cars"]["incorrect"] += 1
            
            bar.update(count)
            
    test_acc = 100.0 * (correct / total)
    print("")
    print('The accuracy on the test dataset:', (test_acc))
    return [correct, total, test_acc]

In [ ]:
HighestAccuracy = 0
ImageSize = 0

for i in range(224, 311):
  print(i)
  test_dataset = TEST_DATASET
  test_transform = transforms.Compose([transforms.Resize(i),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

  dataset = torchvision.datasets.ImageFolder(root=test_dataset, transform = test_transform)
  testloader = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle=False, num_workers = 2)

  model = torch.load(MODEL)
  model.eval()
  info = EvaluateModel(model)
  if info[2] > HighestAccuracy:
    HighestAccuracy = info[2]
    ImageSize = i

print("")
print("The best accuracy is " + str(round(HighestAccuracy, 2)) + "% and the image size is " + str(ImageSize))

In [ ]:
hashMap = {}

In [ ]:
test_dataset = TEST_DATASET
test_transform = transforms.Compose([transforms.Resize(ImageSize),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dataset = torchvision.datasets.ImageFolder(root=test_dataset, transform = test_transform)
testloader = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle=False, num_workers = 2)

In [ ]:
model = torch.load(MODEL)
model.eval()
EvaluateModel(model)

In [ ]:
for category in hashMap:
  correct, incorrect = hashMap[category]['correct'], hashMap[category]['incorrect']
  accuracy = (correct / (correct + incorrect)) * 100
  accuracy = round(accuracy, 2)
  print(f"{category} : {accuracy}%     - correct {correct}   incorrect {incorrect}")